In [1]:
import numpy as np
import math
from functools import reduce
import time
import sys
import os
import numpy as np

path_project = "\\".join(os.getcwd().split("\\")[:-1])
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, path_project)
from pathlib import Path
import nidaqmx

def lcm(a, b):
    """Calculate the least common multiple of two numbers."""
    return abs(a * b) // math.gcd(a, b)


def lcm_of_list(numbers):
    """Find the LCM of a list of numbers."""
    return reduce(lcm, numbers)

def seqtime(seq_tb):
    return np.sum([pulse[-1] for pulse in seq_tb])

# some constants
Hz = 1e-9 # GHz
kHz = 1e-6 # GHz
MHz = 1e-3 # GHz
pi = np.pi


In [2]:
import nidaqmx
from nidaqmx.constants import TerminalConfiguration, VoltageUnits, Edge, AcquisitionType, READ_ALL_AVAILABLE
from nidaqmx.stream_readers import AnalogSingleChannelReader

from hardware import config as hcf
from hardware.hardwaremanager import HardwareManager
from hardware.pulser.pulser import (
    OutputState,
    TriggerStart,
    TriggerRearm,
    HIGH,
    LOW,
    INF,
    REPEAT_INFINITELY
)
timebase = lcm_of_list(
    [hcf.VDISYN_timebase, hcf.SIDIG_timebase, hcf.PS_timebase, hcf.RSRF_timebase]
)

hm = HardwareManager()
# adds synthesizer, laser, and pulse generator 
# hm.add_default_hardware()


In [3]:
# load the hardware manager
# # add some devices to the server (if they aren't already added) ##################################################
import hardware.config as hcf
from hardware.hardwaremanager import HardwareManager
from pathlib import Path
inserv = HardwareManager()
HERE = Path(path_project)
# pulse generator
# 

NI_ch_APD = "/Dev1/ai16"
NI_ch_Clock = "/Dev1/PFI14" # clock source
NI_ch_Trig = "/Dev1/PFI9" # clock gate
NI_ch_Ref_clock= "/Dev1/PFI14"

ch_ai = NI_ch_APD
ch_clock = NI_ch_Clock
ch_trig = NI_ch_Trig
ch_ref_clock = NI_ch_Ref_clock
sampling_rate = 0.5E6 # 500kHz .max ext clock rate of NI6343


try:
    inserv.add(
            'daq', 
            HERE / 'hardware' / 'daq' / 'nidaq.py', 
            'DataAcquisition', 
            [ch_ai, ch_clock, ch_trig, sampling_rate, ch_ref_clock]
        )
except Exception as ee:
    print(ee)

In [4]:
hm.daq.config_readtask(1000, min_volt=-10.0, max_volt=10.0)

Reference clock source: /Dev1/PFI14


Task(name=_unnamedTask<0>)